# Simulering av aksjonspotensialet med virrevandring av ioner

Prosjekt 3 i TMA4320 våren 2019 - av Thorvald Ballestad, Jonas Bueie og Herman Sletmoen (gruppe 3)

I denne prosjektoppgaven simulerer vi aksjonspotensialet i celler.
Aksjonspotensialet er raske potensialvariasjoner mellom innsiden og utsiden av celler og som gjerne forplanter seg som et signal langs nerveceller.
Mekanismen er en konsekvens av ioner som beveger seg inn og ut av celler gjennom cellemembranen i en diffusjonsprosess, påvirket av et elektrisk potensiale.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import constants as consts

# In case reader has old numpy version without heaviside
try:
    heaviside = np.heaviside
except:
    def heaviside(x1, x2):
        return np.where(x1 < 0, 0, np.where(x1 > 0, 1, x2))

In [ ]:
# Setup plotting parameters
newparams = {'axes.labelsize': 15,
             'axes.linewidth': 1,
             'lines.linewidth': 1.5, 
             'figure.figsize': (16, 8),
             'ytick.labelsize': 15,
             'xtick.labelsize': 15,
             'ytick.major.pad': 5,
             'xtick.major.pad': 5,
             'legend.fontsize': 15,
             'legend.frameon': True, 
             'legend.handlelength': 1.5,
             'axes.titlesize': 20,
             'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)


## Oppgave 2 - diffusjonslikningen

Diffusjonslikningen i én dimensjon uttrykker at den romlige distribusjonen $\phi = \phi(x, t)$ over posisjonen $x$ av en substans som har diffundert i en tid $t$ oppfyller
$$\frac{\partial \phi(x, t)}{\partial t} = \frac{\partial}{\partial x} \left(D(x) \frac{\partial \phi(x, t)}{\partial x}\right),$$
der $D(x)$ er en posisjonsavhengig diffusjonskoeffisient.

### Oppgave 2.1 - eksponensiell løsning

Når $D(x) = D$ er posisjonsuavhengig, reduseres diffusjonslikningen til 
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2}.$$
Da er
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$
en løsning av diffusjonslikningen, siden
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2} = \left(\frac{(x-x_0)^2}{4Dt^2} - \frac{1}{2t}\right) \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}.$$

### Oppgave 2.2.1 - diffusjon fra initiell punktsamling

Vi skal nå å finne distribusjonen $\phi(x, t)$ ved et vilkårlig tidspunkt for en initiell punktsamling i $x_0$
$$\phi(x, 0) = \delta(x - x_0) = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-i w (x - x_0)} \mathrm{d} w,$$
der $\delta(x)$ er Diracs deltafunksjon.

Vi antar at distribusjonen i en gitt posisjon $x$ kan taylorutvikles om starttidspunktet $t = 0$ som
$$\phi(x, t) = \sum_{n=0}^{\infty} \frac{t^n}{n!} \frac{\partial^n \phi}{\partial t^n}(x, 0).$$
Den tidsderiverte av første orden ved et vilkårlig tidspunkt er gitt direkte ved diffusjonslikningen som
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2}.$$
Ved å utnytte diffusjonslikningen, og å bytte derivasjonsrekkefølge, finner vi den tidsderiverte av andre orden
$$\frac{\partial^2 \phi(x, t)}{\partial t^2} = \frac{\partial}{\partial t} \left(D \frac{\partial^2 \phi(x, t)}{\partial x^2}\right) = D \frac{\partial^2}{\partial x^2}\frac{\partial \phi(x, t)}{\partial t} = D^2 \frac{\partial^4 \phi(x, t)}{\partial x^4}.$$
Argumentet kan gjentas flere ganger, og vi får generelt
$$\frac{\partial^n \phi(x, t)}{\partial t^n} = D^n \frac{\partial^{2 n} \phi(x, t)}{\partial x^{2 n}}.$$

Ved $t = 0$ er dermed
$$\frac{\partial^n \phi}{\partial t^n}(x, 0) = D^n \frac{\partial^{2 n}}{\partial x^{2 n}} \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-i w (x - x_0)} \mathrm{d} w = D^n \frac{1}{2 \pi} \int_{-\infty}^{+\infty} \frac{\partial^{2 n}}{\partial x^{2 n}} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} (-Dw^2)^{n} e^{-i w (x - x_0)} \mathrm{d} w.$$
Når vi setter dette inn i taylorrekken, finner vi
$$\phi(x, t) = \frac{1}{2 \pi} \sum_{n=0}^{\infty} \frac{t^n}{n!} \int_{-\infty}^{+\infty} (-Dw^2)^{n} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} \sum_{n=0}^{\infty} \frac{(-D w^2 t)^n}{n!} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-D w^2 t} e^{-i w (x - x_0)} \mathrm{d} w,$$
der vi har byttet rekkefølge på integrasjonen og summasjonen, og gjenkjent taylorrekken til eksponensialfunksjonen.
Det siste integralet kan skrives som
$$\phi(x, t) = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-(w \sqrt{D t} + \frac{i(x-x_0)}{2\sqrt{Dt}})^2} e^{-\frac{(x-x_0)^2}{4Dt}} \mathrm{d}w.$$

For å løse integralet, innfører vi $z = w \sqrt{Dt} + \frac{i(x-x_0)}{2\sqrt{Dt}}$, slik at $\mathrm{d}w = \mathrm{d}z/\sqrt{Dt}$.
Dette gir
$$\phi(x, t) = \frac{1}{2 \pi \sqrt{Dt}} e^{-\frac{(x-x_0)^2}{4Dt}} \int_{-\infty}^{+\infty} e^{-z^2} \mathrm{d}z.$$
Integralet i denne likningen har verdien $\sqrt{\pi}$, og vi får til slutt
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}.$$

### Oppgave 2.2.2 - fysisk tolkning av diffusjonskoeffisienten

Med $\sigma^2 = 2 D t$, er løsningen 
$$\phi = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(x - x_0)^2}{2 \sigma^2}},$$
som tilsvarer sannsynlighetsfordelingen til en normalfordeling med forventningsverdi $x_0$ og varians $\sigma^2$.
Siden variansen vokser lineært med tiden og beskriver spredningen til fordelingen, er diffusjonskoeffisienten $D$ et mål på hvor raskt substansen diffunderer.

### Oppgave 2.2.3 - diffusjon fra vilkårlig initiell spredning 

Diffusjonslikningen er lineær, slik at en lineær kombinasjon av flere løsninger også løser likningen.
Vi kan, ved hjelp av den fundamentale løsningen vi fant i oppgave 2.2.1, utnytte denne egenskapen til å uttrykke distribusjonen ved et vilkårlig tidspunkt $t$, gitt en generell initiell fordeling $\phi(x, 0) = f(x).$

Dersom
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$
løser diffusjonslikningen med $\phi(x, 0) = \delta(x - x_0),$ vil lineærkombinasjonen
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} \sum_{n=-\infty}^{+\infty} c_n e^{-\frac{(x - x_n)^2}{4 D t}}$$
være en løsning med initialbetingelsen $\phi(x, 0) = \sum_{n=-\infty}^{+\infty} c_n \delta(x - x_n),$ og integralet
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} \int_{-\infty}^{+\infty} f(y) e^{-\frac{(x - y)^2}{4 D t}} \mathrm{d}y$$
vil være en løsning med en generell initiell distribusjon $\phi(x, 0) = \int_{-\infty}^{+\infty} f(y) \delta(x - y) \mathrm{d}y = f(x).$
Summasjonstegnet er enkelt og greit erstattet med et integral, mens $f(y) \mathrm{d}y$ spiller rollen til konstantene $c_n$.

## Oppgave 3 - virrevandring uten potensiale

I denne oppgaven lar vi 1000 partikler, som opprinnelig befinner seg samlet i origo, gjennomgå en virrevandringsprosess over 100 tidssteg.
Ved hvert tidssteg hopper hver partikkel enten til høyre eller venstre med like stor sannsynlighet.
Til slutt viser vi posisjonsfordelingen til alle partiklene i et histogram, og sammenlikner fordelingen med en normalkurve tilpasset dataene.

In [ ]:
# Random walk without potential
N = 1000
n_steps = 100

plt.title("Distribution of %d particles after %d random walk steps from origin" % (N, n_steps))
plt.xlabel("Position")
plt.ylabel("Proportion of particles")
    
# Simulate random walk
# Particles begin at x=0. For each
# particle +1 or -1 is added n_steps times,
# with equal chance of it being +1 or -1.
pos = np.zeros(N)
steps = np.random.randint(0, 2, size=(n_steps, N))*2 - 1 # Generate random sequence of +1 and -1
step  = np.sum(steps, axis=0)
pos += step
plt.hist(pos, normed = True, label="Simulated distribution", bins=np.arange(np.min(pos), np.max(pos), 2))

# Fit normal curve
mean, stddev = norm.fit(pos)
x = np.linspace(*plt.xlim(), 100)
p = norm.pdf(x, mean, stddev)
plt.plot(x, p, label="Fitted normal distribution")

# Info about fitted curve
textstr = '\n'.join((
    r'$\mu=%.2f$' % (mean),
    r'$\sigma^2=%.2f$' % (stddev**2)))

text_box_props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(-25, 0.03, textstr, fontsize=20,
        verticalalignment='top', bbox=text_box_props)

plt.legend()
plt.show()

Fra teorien i seksjon 1 i oppgaveteksten vet vi at virrevandring med infinitesimale tids- og posisjonssteg svarer til diffusjon, som beskrevet av diffusjonslikningen.<sup>[1]</sup>
Videre er det vist i oppgave 2 at den forventede posisjonsfordelingen til en samling partikler som diffunderer fra samme utgangspunkt, etter en tid $t$ er normalfordelt om startposisjonen med en varians proporsjonal med $t$.
Vi ser av figuren at partikkelfordelingen fra den simulerte virrevandringen og den tilpassede normalkurven samsvarer godt med hverandre og denne teorien.

## Oppgave 5 - virrevandring i tidsuavhengig potensiale

I denne oppgaven simulerer vi en tilsvarende virrevandringsprosess som i oppgave 3, men her i et potensiale der sannsynligheten for å hoppe til venstre eller høyre fra et punkt avhenger av potensialet rundt punktet.
Ved hjelp av statistisk mekanikk kan det vises at sannsynligheten for at en partikkel beveger seg til høyre fra posisjonen $x$ i en slik virrevandringsprosess er
$$P^+ = \frac{1}{1 + e^{-\beta (V(x-h)-V(x+h))}},$$
der $\beta = 1 / k_B T$ er invers termisk energi og $V(x \mp h)$ er partikkelens potensielle energi i nabopunktene.<sup>[1]</sup>
Sannsynligheten for å hoppe til venstre er da
$$P^- = 1 - P^+.$$
Simuleringen vil gjøres med flere ulike potensialer, men ellers samme parametere og initialbetingelse som i oppgave 3.

In [ ]:
def P_right(x, V, betak = 1):
    """Find probability of going right at a given position given a potential function
    Parameters:
     x     : ndarray with positions of which we want probability
     V     : potential function
     betak : value of beta * k, default = 1
    Returns:
     ndarray of same dimention as x"""
    ratio = np.exp(-betak * (V(x-1) - V(x+1)))
    return 1/(1 + ratio)

def simulate(pos, V, n_steps, betak = 1):
    """Simulate random walk in time-independent potential
    Parameters:
     pos    : ndarray with position of particles to be simulated
     V      : potential function
     n_steps: number of time steps to be simulated
     betak  : value of beta * k, default = 1
    Returns:
     ndarray with positions of particles after simulation"""
    pos = np.copy(pos) # Do not change the input array
    x = np.arange(-n_steps, +n_steps + 1) # All possible positions in simulation
    
    for t in range(n_steps): # Go through time
        prob_right_by_pos = P_right(x, V, betak)
        prob_right_by_ion = prob_right_by_pos[pos - x[0]]
        
        step = np.where(np.random.rand(len(pos)) <= prob_right_by_ion, +1, -1)
        pos += step
    
    return pos

def _scale_vals_into_range(rmin, rmax, vals):
    """Helper function to scale vals so that min(vals)=rmin and max(vals)=rmax"""
    vmin = np.min(vals)
    vmax = np.max(vals)
    return rmin + (vals - vmin) / (vmax - vmin) * (rmax - rmin)

def simulate_multiple_kbetas(V, kbetas, N=1000, n_steps=200, potential_desc=None, subplots=True):
    """Simulate and plot random walk for multiple kbeta-values
    Parameters:
     V      : potential function
     kbetas : list-like iterable with kbeta-values
     N      : number of particles, default=1000
     n_steps: number of time steps, default=200
     potential_desc: string with name of the potential
    """
    title_base = "Distribution of %d particles after %d random walk steps from origin" % (N, n_steps)
    if potential_desc is not None:
        title_base += " in %s potential" % potential_desc
        
    starting_pos = np.zeros(N, dtype=int)
    x = np.arange(-n_steps, +n_steps + 1)
    bins = np.arange(-n_steps, +n_steps + 1, 2) # Bin width 2 to cover alternating holes
    for kbeta in kbetas:
        pos = simulate(starting_pos, V, n_steps, kbeta)

        title = title_base + " ($k \\beta = %r$)" % kbeta
        plt.title(title)
        plt.xlabel("Position")
        plt.ylabel("Proportion of particles")
        plt.xlim(-n_steps, +n_steps) # Common for all plots

        fracs, _, _ = plt.hist(pos, bins=bins, normed=True, label="Distribution, $k \\beta = %r$" % kbeta)
        fmax = np.max(fracs)
        if subplots:
            v = V(x)
            v = _scale_vals_into_range(0, 0.5 * fmax, v)
            plt.plot(x, v, label="Potential (qualitative)") # Plot potential qualitatively
            plt.legend()
            plt.show()
            
    if not subplots:
        v = V(x)
        v = _scale_vals_into_range(0, 0.5 * plt.ylim()[1], v)
        plt.plot(x, v, label="Potential (qualitative)")
        plt.legend()
        plt.show()

### Oppgave 5.1 - lineært potensiale

Først simulerer vi virrevandringen til partikler med lineær potensiell energi
$$V(x) = k x,$$
for en konstant $k$, som representerer en konstant kraft på partiklene.

In [ ]:
def V_linear(x):
    return x

simulate_multiple_kbetas(V_linear, (0.1, 0.8, 1.5), potential_desc="linear", subplots=False)

Figuren viser fordelingen av partikler etter diffusjonsprosessen, samt en kvalitativ skisse av den lineære potensialfunksjonen, for tre ulike verdier av $k\beta$. 
Det fremkommer av figurene at partiklene vandrer lengre langs kraftfeltet desto større forholdet $k \beta$ mellom den potensielle energien og den termiske energien er.
Den fysiske forklaringen på dette er at lavere termisk energi tilsvarer færre tilfeldige vibrasjonsbevegelser, slik at partiklene domineres av kraftfeltet.
Når den termiske energien er høy, vil partiklenes tilfeldige vibrasjonsbevegelser øke deres evne til å ignorere kraftfeltet og bevege seg i en tilfeldig retning.
Dette samsvarer matematisk med uttrykket for sannsynligheten $P^+$.

### Oppgave 5.2 - bokspotensiale

Vi simulerer nå virrevandringsprosessen til partikler med potensiell energi
$$V(x) = \begin{cases} k & -3h < x < +3h \\ 0 & \text{ellers} \\ \end{cases}.$$
Disse partikklene beveger seg uten påvirkning av krefter der potensialet er flatt, og utsettes for krefter i overgangsområdene mellom disse flate regionene.

In [ ]:
def V_membrane(x, d = 3):
    return np.where(np.abs(x) < d, 1, 0)

simulate_multiple_kbetas(V_membrane, (0.5, 1.0, 1.5), potential_desc="box")

Figurene viser posisjonsfordelingen etter diffusjonsprosessen, sammen med det tilhørende bokspotensialet. 
Det fremkommer at færre partikler befinner seg i området med høy potensiell energi, jo lavere den termiske energien er.
Dette kan igjen forklares med at lavere termisk energi øker dominansen til kraftfeltet og dermed sjansen for at partiklene hopper ned fra platået i midten, samtidig som det også blir vanskeligere for partikler å hoppe opp på platået.

### Oppgave 5.3 - lineært potensiale med flate endestykker

Her simulerer vi virrevandringsprosessen til partikler med potensiell energi
$$V(x) = \begin{cases} -k & x \leq -3h \\ k \left(-1 + 2 \frac{x+3h}{6h} \right) & -3h < x < +3h \\ +k & x \geq +3h \\ \end{cases}.$$
Disse partiklene påvirkes av en konstant kraft i det avgrensede området mellom $-3h$ og $+3h$, men er ellers upåvirkede.

In [ ]:
def V_linear_edge(x):
    return np.where(np.abs(x) < 3, -1 + 2*(x + 3)/6, np.sign(x))

simulate_multiple_kbetas(V_linear_edge, (0.1, 0.8, 1.5), potential_desc="restricted linear")

Figurene viser resultatet av diffusjonsprosessen for det avgrensede lineære potensialet, for tre ulike verdier av $k\beta$.
Liten termisk energi gir her en posisjonsfordeling med de fleste partiklene i området med lavere potensiell energi, mens stor termisk energi gir en posisjonsfordeling som er mer upåvirket av potensialet.
Nok en gang er forklaringen at potensialet påvirker partiklene mer når den potensielle energien er stor sammenliknet med den termiske energien.

## Oppgave 7 og 8 - virrevandring av ioner i cellemembran med tidsavhengig potensiale og simulering av aksjonspotensialet

I disse oppgavene modellerer vi potensialforskjellen over membranen i en enkel, endimensjonal celle og studerer aksjonspotensialfenomenet.

Vi simulerer en situasjon der natrium- og kaliumioner først plasseres inne i og utenfor cellen, før de slippes løs og utsettes for en potensialpåvirket virrevandringsprosess der de kan vandre inn og ut av cellen gjennom hver sine porter i cellemembranen.
I portene opplever ionene hvert sitt flate potensiale $V_{Na}$ og $V_K$ i tillegg til et felles lineært tidsavhengig potensiale $V(t)$ over cellemembranen.
Utenfor membranområdet er potensialet flatt og ionene upåvirket av krefter.
Membranen antas å oppføre seg som en kondensator, slik at membranpotensialet bestemmes av forskjellen i ladningskonsentrasjonen $\Delta [Q] = [Q]_{in} - [Q]_{out}$ mellom innsiden og utsiden av cellen som $V(t) = \Delta [Q] / C_c$, der $C_c$ er en konstant konsentrasjonskapasitans for cellemembranen.
I modellen antar vi at det kun er ionekonsentrasjonen $[Q]_{in}$ på innsiden av cellen som varierer med tiden, mens utsiden av cellen betraktes som store omgivelser med en konstant ionekonsentrasjon $[Q]_{out}$.
I alle tilfeller starter simulasjonen med mye mer natrium utenfor cellen enn innenfor, og mye mer kalium innenfor cellen enn utenfor.
Disse initielle konsentrasjonsgradientene ønsker å sende natrium inn i cellen og kalium ut, men potensialet må tas med i betrakning for å bestemme den endelige strømningen.
Selv om vi behandler ionene som enkeltioner i simuleringen, representerer hver av dem i realiteten en ionekonsentrasjon.

In [ ]:
# ---- Global constants -----
T = 273 + 37 # K, body temperature
beta = 1 / (consts.k*T)
C = 0.07e3 # e mM / V

gate_lower_v_limit = -70e-3 # Volts
gate_upper_v_limit = +30e-3 # Volts

V_beta_closed = 50
V_closed = V_beta_closed / beta # Closed gate potential for both Na and K

cons_scaling_factor = 0.1 # mM, the concentration each particle represents

def inside_concentration(pos):
    """Find concentration of particles inside the cell"""
    n_inside = len(pos[pos > 1])
    return n_inside * cons_scaling_factor

def outside_concentration(pos):
    """Find concentration of particles outside the cell"""
    return inside_concentration(-pos)

def V_diff(pos_Na, pos_K, conc_out, C):
    """Returns the voltage difference across the membrane"""
    conc_in = inside_concentration(pos_Na) + inside_concentration(pos_K)
    conc_diff = conc_in - conc_out
    return conc_diff / C

def simulate_v2(V0_Na_beta=1, V0_K_beta=1, action_potential=False, pump=False, n_steps=1000, L=50):
    """Simulate random walk of Na+ and K+ across a cell membrane
    Parameters:
     V0_Na_beta      : Gate potential for open-state Na-gate
     V0_K_beta       : Gate potential for open-state K-gate
     action_potential: Include the action potential in the simulation, default=False
     pump            : Include ion-pump in the simulation, default=False
     n_steps         : Number of time steps to simulate, default=1000
     L               : Length of the system, default=50
    Returns:
     v_diffs : list with voltage differences for each time step
     nnaouts : list with number of Na outside for each time step
     nkouts  : list with number of K outside for each time step
     nnains  : list with number of Na inside for each time step
     nkins   : list with number of K inside for each time step"""
    
    V0_Na = V0_Na_beta / beta
    V0_K = V0_K_beta / beta

    # Put ions in start positions (negative positions outside, positive positions inside)
    N_Na_inside = 50
    N_Na_outside = 1450
    N_K_inside = 1400
    N_K_outside = 50
    N_Na = N_Na_inside + N_Na_outside
    N_K = N_K_inside + N_K_outside
    
    pos_Na = np.append(np.full(N_Na_outside, -L//4), np.full(N_Na_inside, +L//4))
    pos_K = np.append(np.full(N_K_outside, -L//4), np.full(N_K_inside, +L//4))
    
    conc_out = outside_concentration(pos_Na) + outside_concentration(pos_K) # Constant throughout sim
      
    if action_potential:
        # Start with gates closed
        V_Na = V_closed
        V_K = V_closed
    else:
        # Start with gates open
        V_Na = V0_Na
        V_K = V0_K
    
    if pump:
        n_Na_per_pumping = 3
        n_K_per_pumping = 2
        min_steps_per_pumping = 10 # Minimum number of time steps between each pumping
        steps_until_pumping = 0 # Because it is possible that there are not enough 
                                # ions at a given time to pump, it is not sufficient
                                # to only check time % min_steps_per_pumping == 0. 
                                # steps_until_pumping makes possible pumping as soon as
                                # there are sufficiently many ions to pump, and then wait 10 steps after that.
                                # The simpler solution of checking the time % min_steps_per_pumping
                                # would potentialy wait even when there are enough ions available.
        
    x = np.arange(-L//2, +L//2 + 1) # All possible positions in simulation
    membrane = V_membrane(x, d=2) # Time-independent term of of the potential
        
    # Stats to be returned
    v_diffs = []
    nnaouts = []
    nkouts = []
    nnains = []
    nkins = []

    for t in range(n_steps): # Go through time
        v_diff = V_diff(pos_Na, pos_K, conc_out, C)
        v_diff_array = v_diff * heaviside(x, 0.5) * consts.e
        
        # Update stats
        v_diffs.append(v_diff)
        nnains.append(len(pos_Na[pos_Na > 1]))
        nnaouts.append(len(pos_Na[pos_Na <= 1]))
        nkins.append(len(pos_K[pos_K > 1]))
        nkouts.append(len(pos_K[pos_K <= 1]))
        
        if action_potential:
            if v_diff < gate_lower_v_limit:
                V_Na = V0_Na # Open sodium gate
                V_K = V_closed # Close potassium gate
            elif v_diff > gate_upper_v_limit:
                V_Na = V_closed # Close sodium gate
                V_K = V0_K # Open potassium gate
        
        # Do this for both Na and K
        for V, pos in zip((V_Na, V_K), (pos_Na, pos_K)):
            v_total = v_diff_array + V * membrane # Total potential experienced by ion
            prob_right_by_pos = 1 / (1 + np.exp(beta * (np.roll(v_total, -1) - np.roll(v_total, +1))))
            prob_right_by_pos[-1] = 0 # Force rightmost ions to jump left
            prob_right_by_pos[0] = 1 # Force leftmost ions to jump right
            prob_right_by_ion = prob_right_by_pos[pos - x[0]]

            # Jump left/right with calculated probabilities
            step = np.where(np.random.rand(len(pos)) <= prob_right_by_ion, +1, -1)
            pos += step
            
        if pump:
            steps_until_pumping -= 1
            if steps_until_pumping <= 0:
                # Pump the ions closest to the membrane
                pos_Na.sort() # Sort ascending
                pos_K[::-1].sort() # Sort descending
                index_Na = np.argwhere(pos_Na > 0)[:n_Na_per_pumping] # Pump ions closest to membrane
                index_K = np.argwhere(pos_K < 0)[:n_K_per_pumping] # Pump ions closest to membrane
                if len(index_Na) == n_Na_per_pumping and len(index_K) == n_K_per_pumping: # Check that there are enough ions
                    pos_Na[index_Na] = -1 # Pump Na ions out, place them just outside
                    pos_K[index_K] = +1 # Pump K ions in, place them just inside
                    steps_until_pumping = min_steps_per_pumping
                
    return v_diffs, nnaouts, nkouts, nnains, nkins

def _get_title(action_potential, pump, V0_Na_beta, V0_K_beta):
    """Generate title for simulate_v2_plot"""
    title = "Ion amounts and membrane voltage in cell simulation"
    if not action_potential and not pump:
        title += " without action potential and pump"
    elif action_potential and not pump:
        title += " with action potential and without pump"
    elif action_potential and pump:
        title += " with action potential and pump"
    title += " ($\\beta V_{Na}^0 = %r ,\\,\\, \\beta V_{K}^0 = %r$)" % (V0_Na_beta, V0_K_beta)
    return title
    
    
def simulate_v2_plot(V0_Na_beta, V0_K_beta, action_potential=False, pump=False,
                     plot_high_amounts=True, plot_low_amounts=True, n_steps=1000):
    """Simulate using simulate_v2 and plot"""
    
    v_diffs, nnaouts, nkouts, nnains, nkins = simulate_v2(V0_Na_beta, V0_K_beta, action_potential, pump, n_steps=n_steps)
    
    fig = plt.figure()
    title = _get_title(action_potential, pump, V0_Na_beta, V0_K_beta)

    # Create subplots according to desired plotting parameters
    if plot_high_amounts or plot_low_amounts:
        ax1 = fig.add_subplot(2, 1, 1)
        ax2 = ax1.twinx()
        ax3 = fig.add_subplot(2, 1, 2)
        plt.setp(ax1.get_xticklabels(), visible=False)
    else:
        ax3 = fig.add_subplot(1, 1, 1)
    fig.suptitle(title, size=20)
    ax3.set_xlabel("Time (time steps)")
    
    if plot_low_amounts:
        ax2.set_ylabel("Ion amount (low)")
        ax2.plot(nnains, "-C0", label="Na amount inside (low)", alpha=0.2)
        ax2.plot(nkouts, "-C1", label="K amount outside (low)", alpha=0.2)
    if plot_high_amounts:
        ax1.set_ylabel("Ion amount (high)")
        ax1.plot(nnaouts, "-C0", label="Na amount outside (high)", alpha=1.0)
        ax1.plot(nkins, "-C1", label="K amount inside (high)", alpha=1.0)
        
    ax3.set_ylabel("Membrane voltage [V]")
    ax3.plot(v_diffs, "-C2", label="Membrane voltage")
    
    
    if action_potential:
        ax3.axhline(gate_upper_v_limit, color="red", linestyle="dashed", label="Upper gate voltage limit")
        ax3.axhline(gate_lower_v_limit, color="red", linestyle="dashed", label="Lower gate voltage limit")
    
    if plot_low_amounts or plot_high_amounts:
        h1, l1 = ax1.get_legend_handles_labels()
        h2, l2 = ax2.get_legend_handles_labels()
        ax1.legend(h1+h2, l1+l2, loc="upper right")
    ax3.legend(loc="best")
    fig.tight_layout()
    fig.subplots_adjust(top=0.92)
    plt.show()

### Oppgave 7.1 - virrevandring i cellemembran med identiske ionekanalpotensialer

I første omgang gjøres simulasjonen med tidsuavhengige og like verdier for portpotensialene $V_{Na} = V_K = V_{Na}^0 = V_K^0$.

In [ ]:
simulate_v2_plot(0, 0)
simulate_v2_plot(1, 1)
simulate_v2_plot(3, 3)

Figurene viser at prosessene innstiller seg mot en likevekt, der det strømmer like mye ioner av hver type inn og ut av cellen.
Da endrer ikke ionekonsentrasjonen på innsiden seg lenger, og spenningen over membranen stabiliserer seg rundt $0 \, \mathrm{V}$.
Vi forklarer tidsutviklingen i prosessen under.

1. I starten strømmer natrium raskt inn i cellen på grunn av både den negative membranspenningen og konsentrasjonsgradienten over cellen.
Kaliumionene er mer tilbakeholdne, da diffusjonen fra konsentrasjonsgradienten her må jobbe mot spenningsforskjellen.
2. Etter at det er like mange ioner inni og utenfor cellen, fortsetter natrium å strømme inn, men da kun grunnet den fortsatt negative membranspenningen, som dominerer over den motvirkende diffusjonseffekten fra konsentrasjonsgradienten.
3. Når det har strømmet inn så mye natrium slik at ionekonsentrasjonen inni cellen er det samme som den var på utsiden ved start, er det ingen spenning over membranen, og likevekt er oppnådd.
En eventuell strømning som forstyrrer membranspenningen fra likevekt, vil alltid være opphav til en ny membranspenning som forårsaker en tilbakestrømning som reverserer den opprinnelige forstyrrelsen.
Dette stemmer godt med det velkjente prinsippet til Le Chatelier.
4. Etter at likevekt er oppnådd, er det like enkelt for et gitt natrium- eller kaliumion å strømme gjennom membranen.
Da strømmer natrium- og kaliumionene hver sin vei gjennom membranen kontinuerlig, slik at likevektspenningen opprettholdes, mens nettostrømningen gradvis avtar.
5. Etter svært lang tid er det ingen netto strømning av hverken natrium- eller kaliumioner over membranen, og prosessen dør ut.
Merk at dette kun innebærer en like stor strøm av natrium inn i cellen som ut, og tilsvarende for kalium.

Vi ser også at svingningene rundt likevekt er større, jo større den termiske energien er i forhold til portpotensialene.
Dette svarer igjen til at de økte tilfeldige vibrasjonsbevegelsene klarer å dytte systemet lenger unna likevekt.

Kort oppsummert er det utenkelig at denne prosessen, der ioner overlates til naturlige krefter uten påvirkning utenfra, skulle tendere mot noe annet enn stabilitet og likevekt.
Siden portpotensialene er like så de to ionetypene kan flyte like enkelt over membranen, er det i prinsippet ingen forskjell på de to.
Modellen kunne derfor like gjerne beskrevet strømningen av én type ladningsbærer rundt i et metall.
Fra elektromagnetismen er det en kjent sak at ladningene her vil distribuere seg slik at det elektriske potensialet er likt overalt i metallet.
Om vi skulle hatt en prosess som ikke stabiliserte seg, måtte vi påvirket den ved å tilføre energi fra en ekstern kilde.

### Oppgave 7.2 - virrevandring i cellemembran med ulike ionekanalpotensialer

Vi gjennomfører nå en tilsvarende simulering som i oppgave 7.1, men her med en forskjell i portpotensialene $V_{Na} = V_K$.

In [ ]:
simulate_v2_plot(10, 1)
simulate_v2_plot(1, 10)

Også her viser figurene at vi får en likevekt, men her stanser nettostrømningen mye raskere, og membranspenningen stabiliserer seg rundt en positiv eller negativ verdi, til forskjell fra i oppgave 7.1.
Ionet med portpotensiale så sterkt som $V^0 = 10 / \beta$ stenger effektivt porten til det aktuelle ionet, mens det andre ionet strømmer over membranen på samme måte som før.
Dette betyr at det sterke portpotensialet overvinner de naturlige diffusjonskreftene, som stemmer godt overens med den eksponensielle avhengigheten av forholdet mellom portpotensialet og den termiske energien i uttrykket for $P^+$.

Dermed strømmer ionet med åpen port inn i cellen, inntil strømningseffekten fra membranspenningen akkurat motvirker strømningseffekten fra konsentrasjonsgradienten og likevekt er oppnådd.
Da blir membranspenningen positiv eller negativ, avhengig av skjevheten i distribusjonen av den andre ionetypen.
Riktignok ser vi at et og annet ion av den andre typen strømmer gjennom den nesten blokkerte porten, men modellen må studeres over svært lang tid for at dette skal bli en del av historien, noe som er irrelevant her.

Fraværet av ytre påvirkninger, gjør igjen at det er fullstendig ufysikalsk at prosessene skal tendere mot not annet enn likevekt.
Avhengig av om begge eller kun én av ionetypene har mulighet til å gå gjennom cellemembranen, får vi altså en naturlig selvregulerende likevektsituasjon der membranspenningen stabiliserer seg og det strømmer like mye ioner av hver type hver vei gjennom membranen.

### Oppgave 8.1 - aksjonspotensialet uten ionepumpe

I biologiske celler er ioneporter spenningskontrollerte.
Det betyr at de åpnes og lukkes avhengig av membranspenningen.
I denne oppgaven utvider vi modellen fra oppgave 7 ved å studere denne effekten.
Ved ethvert tidspunkt lar vi en av portene være åpen og den andre lukket ved å sette det ene portpotensialet til $V_{open} = 1 / \beta$, som før, og det andre til $V_{closed} = 50 / \beta$, for å være bombesikre på at porten lukkes når membranspenningen overgår en gitt gulv- eller takverdi.

In [ ]:
simulate_v2_plot(1, 1, action_potential=True)

Fra figuren ser vi at den initielle ionedistribusjonen sørger for å åpne natriumporten fra starten av, mens kaliumporten stenges.
Da får vi en reprise av natriumstrømningen inn i cellen med kaliumporten stengt, som i oppgave 7.2.
Men her når membranspenningen takverdien før den stabiliserer seg, som stenger natriumporten og åpner kaliumporten.
Da gjentas kaliumstrømningen med natriumporten stengt fra oppgave 7.2.
Spenningen stabiliserer seg igjen, som i oppgave 7.2, men ved en høyere verdi enn i oppgave 7.2, takket være den tidligere natriumstrømningen.
Derfor dør prosessen ut.
Noen ganger nås gulvspenningen en andre og tredje gang ved tilfeldigheter.
Da gjentas pulsen, men den stabiliserer seg ved en noe høyere spenningsverdi enn i forrige puls, slik at gulvspenningen ikke nås ytterligere ganger og prosessen til slutt dør helt ut.

### Oppgave 8.2 - aksjonspotensialet med ionepumpe

I biologiske celler er det også en ionepumpe, som ved hjelp av en ytre energikilde pumper en mengde kaliumioner inn og en *større* mengde natriumioner ut av cellen.
Vi utvider her modellen fra oppgave 8.1 til å inkludere en slik natrium-kalium-pumpe som kontinuerlig pumper ioner på denne måten, og skal se hvordan dette er den siste nødvendige brikken i puslespillet for å modellere et repeterende aksjonspotensiale.

In [ ]:
simulate_v2_plot(1, 1, action_potential=True, pump=True, n_steps=1000) # See details of the pumping process
simulate_v2_plot(1, 1, action_potential=True, pump=True, n_steps=5000) # See the process over a longer time

Fra figurene ser vi at ionepumpen får aksjonspotensialet til å gå syklisk.
Vi ser fra den første figuren at skjevheten i pumpevolumene av natrium og kalium effektivt sett gjenoppretter startforholdene kontinuerlig gjennom hele prosessen, samtidig som ionekonsentrasjonen på innsiden og membranspenningen senkes, der den i oppgave 8.1 stabiliserte seg.
Dette gjør at membranspenningen når gulvspenningen, der kaliumporten på nytt stenges, mens natriumporten åpnes, og vi får en periodisk gjentakelse av signalet i oppgave 8.1.
Den andre figuren viser at den kontinuerlige gjenopprettelsen av startforholdene, gjør prosessen i stand til å opprettholde seg selv uten å dø ut, slik den gjør i de tidligere oppgavene.
Det er dette periodiske av-og-på-signalet som kalles aksjonspotensialet, og som spiller en sentral rolle i kommunikasjon mellom celler ved hjelp av et signal som forplanter seg langs cellene.

## Referanser
[1] Simensen, Håkon T: *Prosjekt 3 - TMA4320 - Simulating the Action Potential with Random Walk of Ions.*